In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import numpy as np
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io
import cv2
import torch.nn.functional as F
import time
from torchvision.datasets import ImageFolder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
main_folder = '/content/drive/MyDrive/EWU_BHCR'
root_dir = '/content/drive/MyDrive/EWU_BHCR/Dataset'
train_dir = '/content/drive/MyDrive/EWU_BHCR/Dataset/Train'
valid_dir = '/content/drive/MyDrive/EWU_BHCR/Dataset/Test'

In [ ]:
IMG_SIZE = 32
BATCH_SIZE = 32
LOSS_FN = 'CrossEntropyLoss'
OPTIMIZER_LIST = ['adam', 'RMSprop']
DROPOUT_LIST = [.25, .5]
LEAENING_RATE_LIST = [.001, .0001]
EPOCHS = 10

NUM_CLASSES = 50

In [ ]:
df = pd.DataFrame(columns = [
    'IMG_SIZE',
    'BATCH_SIZE',
    'LOSS_FN',
    'OPTIMIZER',
    'DROPOUT_RATIO',
    'LEAENING_RATE',
    'EPOCHS',
    'TRAINABLE_PARAMS',
    'TRAIN_LOSS',
    'TRAIN_ACCURACY',
    'TRAIN_TIME',
    'VALID_ACCURACY',
    'VALID_TIME'
])

In [ ]:
for OPTIMIZER in OPTIMIZER_LIST:
    for DROPOUT in DROPOUT_LIST:
        for LEAENING_RATE in LEAENING_RATE_LIST:
            transform = transforms.Compose([
                transforms.Resize((IMG_SIZE, IMG_SIZE)),
                transforms.ToTensor()
            ])

            train_loader = DataLoader(
                ImageFolder(train_dir,transform=transform),
                batch_size = BATCH_SIZE, shuffle = True
            )

            valid_loader = DataLoader(
                ImageFolder(valid_dir,transform=transform),
                batch_size = BATCH_SIZE, shuffle = True
            )



            class Model(nn.Module):
                def __init__(self):
                  super(Model, self).__init__()
                  self.relu = nn.ReLU()
                  self.avgpool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2,2))
                  self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5, 5), stride=(1, 1))
                  self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1))
                  self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5), stride=(1, 1))
                  self.linear1 = nn.Linear(in_features=120, out_features=84)
                  self.linear2 = nn.Linear(in_features=84, out_features=NUM_CLASSES)
                
                def forward(self, x): # 32x32
                  x = self.avgpool(self.relu(self.conv1(x))) # 28x28x6 => 14x14x6
                  x = self.avgpool(self.relu(self.conv2(x))) # 10x10x16 => 5x5x16
                  x = self.relu(self.conv3(x)) # 1x1x120
                  x = x.reshape(x.shape[0], -1)
                  x = self.relu(self.linear1(x))
                  x = self.linear2(x)
                  return x  


            model = Model().to(device)


            TRAINABLE_PARAMS = sum(p.numel() for p in model.parameters() if p.requires_grad)


            loss_fn = nn.CrossEntropyLoss()

            if OPTIMIZER == 'sgd':
                optimizer = optim.SGD(model.parameters(), lr = LEAENING_RATE)
            elif OPTIMIZER == 'adam':
                optimizer = optim.Adam(model.parameters(), lr = LEAENING_RATE)
            elif OPTIMIZER == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr = LEAENING_RATE)
            elif OPTIMIZER == 'Adadelta':
                optimizer = optim.Adadelta(model.parameters(), lr = LEAENING_RATE)
            elif OPTIMIZER == 'Adamax':
                optimizer = optim.Adamax(model.parameters(), lr = LEAENING_RATE)



            t_loss = 0
            t_acc = 0
            t_time = 0
            v_acc = 0
            v_time = 0

            for epoch in range(EPOCHS):
                train_starts = time.time()
                model.train()
                train_count, correct_preds = 0, 0
                train_loss = 0.
                for i, (images, labels) in enumerate(train_loader):
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    loss = loss_fn(outputs, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    _, preds = torch.max(outputs.data, 1)
                    train_count += labels.size(0)
                    correct_preds += (preds == labels).sum().item()
                    train_loss += loss.item() * labels.size(0)
                train_acc = (correct_preds / train_count) * 100.
                train_loss = train_loss / train_count
                train_ends = time.time()
                train_time = train_ends - train_starts


                valid_starts = time.time()
                model.eval()
                with torch.no_grad():
                    correct_preds, valid_count = 0, 0
                    for images, labels in valid_loader:
                        images = images.to(device)
                        labels = labels.to(device)
                        outputs = model(images)
                        _, preds = torch.max(outputs.data, 1)
                        valid_count += labels.size(0)
                        correct_preds += (preds == labels).sum().item()
                    valid_acc = (correct_preds / valid_count) * 100.
                    valid_ends = time.time()
                    valid_time = valid_ends - valid_starts
                                                
                                                
                    t_loss = train_loss
                    t_acc = train_acc
                    t_time = train_time
                    v_acc = valid_acc
                    v_time = valid_time
                print(f'Epoch: {epoch}, Train_Accuracy: {train_acc:.2f}%, Train_Loss: {train_loss:.2f}, Train_Time: {train_time:.1f}s, Valid_Accuracy: {valid_acc:.2f}%, Valid_Time: {valid_time:.1f}s')
                
            new_row = {
                'IMG_SIZE': IMG_SIZE,
                'BATCH_SIZE': BATCH_SIZE,
                'LOSS_FN': LOSS_FN,
                'OPTIMIZER': OPTIMIZER,
                'DROPOUT_RATIO': DROPOUT,
                'LEAENING_RATE': LEAENING_RATE,
                'EPOCHS': EPOCHS,
                'TRAINABLE_PARAMS': TRAINABLE_PARAMS,
                'TRAIN_LOSS': f'{t_loss:.2f}',
                'TRAIN_ACCURACY': f'{t_acc:.2f}',
                'TRAIN_TIME': f'{t_time:.1f}',
                'VALID_ACCURACY': f'{v_acc:.2f}',
                'VALID_TIME': f'{v_time:.1f}'
            }

            df = df.append(new_row, ignore_index = True)
            df.to_csv('/content/drive/MyDrive/EWU_BHCR/ExperimentsResults/LeNet5.csv', index = False)
            print('\n')
        print('\n\n\n')

Epoch: 0, Train_Accuracy: 12.40%, Train_Loss: 3.36, Train_Time: 3462.9s, Valid_Accuracy: 21.53%, Valid_Time: 864.1s
Epoch: 1, Train_Accuracy: 37.90%, Train_Loss: 2.23, Train_Time: 13.7s, Valid_Accuracy: 43.90%, Valid_Time: 3.2s
Epoch: 2, Train_Accuracy: 51.14%, Train_Loss: 1.72, Train_Time: 13.6s, Valid_Accuracy: 55.03%, Valid_Time: 3.2s
Epoch: 3, Train_Accuracy: 60.35%, Train_Loss: 1.38, Train_Time: 13.6s, Valid_Accuracy: 59.83%, Valid_Time: 3.1s
Epoch: 4, Train_Accuracy: 65.40%, Train_Loss: 1.18, Train_Time: 13.5s, Valid_Accuracy: 67.00%, Valid_Time: 3.2s
Epoch: 5, Train_Accuracy: 69.60%, Train_Loss: 1.03, Train_Time: 13.9s, Valid_Accuracy: 69.73%, Valid_Time: 3.3s
Epoch: 6, Train_Accuracy: 72.81%, Train_Loss: 0.91, Train_Time: 13.5s, Valid_Accuracy: 69.67%, Valid_Time: 3.2s
Epoch: 7, Train_Accuracy: 75.25%, Train_Loss: 0.81, Train_Time: 13.5s, Valid_Accuracy: 72.30%, Valid_Time: 3.2s
Epoch: 8, Train_Accuracy: 77.55%, Train_Loss: 0.75, Train_Time: 13.6s, Valid_Accuracy: 75.20%, Valid